# Day 9 - Part 1

In [ ]:
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        dx = abs(data[i][0] - data[y][0])
        dy = abs(data[i][1] - data[y][1])
        area = dx * dy
        if area > big:
            big = area
            p1, p2 = data[i], data[y]

print(big)

4754817236


# Day 9 - Part 2

In [279]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

class Rect:
    def __init__(self, p1: Point, p2: Point):
        self.left   = min(p1.x, p2.x)
        self.right  = max(p1.x, p2.x)
        self.top    = min(p1.y, p2.y)
        self.bottom = max(p1.y, p2.y)

    def get_area(self):
        return (self.right - self.left + 1) * (self.bottom - self.top + 1)

# --- solution
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

data = [Point(x, y) for x, y in data]
edges = [Rect(data[i], data[i+1]) for i in range(len(data[:-1]))]
edges.append(Rect(data[-1], data[0]))

def overlap(rect: Rect, edge: Rect):
    if rect.right <= edge.left or rect.left >= edge.right:
        return False
    if rect.bottom <= edge.top or rect.top >= edge.bottom:
        return False

    return True

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        rect = Rect(data[i], data[y])
        area = rect.get_area()
        if area <= big:
            continue

        has_overlap = any(overlap(rect, edge) for edge in edges)

        if not has_overlap:
            big = area

big

1568849600

# extra stuff

In [ ]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

class Rect:
    def __init__(self, p1: Point, p2: Point):
        # normalize so origin is always top right
        self.left   = min(p1.x, p2.x)
        self.right  = max(p1.x, p2.x)
        self.top    = min(p1.y, p2.y)
        self.bottom = max(p1.y, p2.y)

    def get_area(self):
        return (self.get_width() + 1) * (self.get_height() + 1)  # inclusive
    
    def get_width(self):
        return self.right - self.left
    
    def get_height(self):
        return self.bottom - self.top
    
    def get_origin(self):
        return Point(self.left, self.top)
    
    def get_points(self):
        # points in clockwise order
        self.top_left     = Point(self.left, self.top)
        self.top_right    = Point(self.right, self.top)
        self.bottom_right = Point(self.right, self.bottom)
        self.bottom_left  = Point(self.left, self.bottom)

        return self.top_left, self.top_right, self.bottom_right, self.bottom_left

    def __repr__(self):
        return f"Rect(P1({self.left},{self.top}) P2({self.right},{self.bottom}))"


def overlap(r1: Rect, r2: Rect):
    overlap_x = r1.left <= r2.right and r1.right >= r2.left
    overlap_y = r1.top <= r2.bottom and r1.bottom >= r2.top

    x, y = None, None

    if overlap_x:
        x = max(r1.left, r2.left) 

    if overlap_y:
        y = max(r1.top, r2.top)

    if overlap_x and overlap_y:
        return Rect(Point(x, y), Point(min(r1.right, r2.right), min(r1.bottom, r2.bottom)))

    return False


# cuts horizontaly (left to right)
def cut_h_segments(slab: Rect, cutter: Rect):
    ov = overlap(slab, cutter)
    cuts = []

    if not ov:
        return None
    
    if ov.left > slab.left:
        cuts.append(Rect(Point(slab.left, ov.top), ov.bottom_left))

    if ov.right < slab.right:
        cuts.append(Rect(ov.top_right, Point(slab.right, ov.bottom)))

    if ov.top > slab.top:
        cuts.append(Rect(slab.top_left, Point(slab.right, ov.top)))

    if ov.bottom < slab.bottom:
        cuts.append(Rect(Point(slab.left, ov.bottom), slab.bottom_right))

    return cuts


# TODO: implement - cuts vertically (top to bottom)
def cut_v_segments(slab: Rect, cutter: Rect):
    pass


# xy
p1 = (2, 1)
p2 = (7, 6)

p3 = (1, 4)
p4 = (5, 9)

points = [p1, p2, p3, p4]
min_x, max_x = min([x for x, _ in points]), max([x for x, _ in points])
min_y, max_y = min([y for _, y in points]), max([y for _, y in points])

grid = [['.' for _ in range(max_x + 1)] for _ in range(max_y + 1)]

pob = [Rect(Point(p1[0], p1[1]), Point(p2[0], p2[1])),
       Rect(Point(p3[0], p3[1]), Point(p4[0], p4[1]))]

# for p in pob:
#     for point in p.points:
#         grid[point.y][point.x] = '#'

# new = cut_h_segments(pob[1], pob[0])

# for p in new:
#     for point in p.points:
#         grid[point.y][point.x] = 'x'


# --- solution
with open("red_tiles.txt", 'r', encoding='utf-8') as f:
    data = [tuple(map(int, line.strip().split(','))) for line in f.readlines()]

data = [Point(x, y) for x, y in data]
edges = [Rect(data[i], data[i+1]) for i in range(len(data[:-1]))]
edges.append(Rect(data[-1], data[0]))

def overlap(rect: Rect, edge: Rect):
    if rect.right < edge.left or rect.left > edge.right:
        return False
    if rect.bottom < edge.top or rect.top > edge.bottom:
        return False

    touching_x = (rect.right == edge.left) or (rect.left == edge.right)
    touching_y = (rect.bottom == edge.top) or (rect.top == edge.bottom)

    if touching_x or touching_y:
        return False

    return True

big = 0
for i in range(len(data)):
    for y in range(i + 1, len(data)):
        rect = Rect(data[i], data[y])
        has_overlap = any(overlap(rect, edge) for edge in edges)

        if not has_overlap:
            area = rect.get_area()
            if area > big:
                big = area
                last = rect

big

1568849600